In [27]:
import cv2
import time
import json
import random
from PIL import Image
import numpy as np
from mss import mss
import serial, time
import binascii

In [48]:
cap = cv2.VideoCapture(0)
sct = mss()

In [52]:
arduino = serial.Serial('/dev/cu.SLAB_USBtoUART', 115200, timeout=.1)
src_dev = 0
w, h = 64, 64
monitor = {'top': 0, 'left': 0, 'width': w, 'height': h}

In [60]:
while True:
    while True:
        rline = arduino.readline().strip()
        if len(rline) > 0:
            print(rline)
        else:
            break
        
    if src_dev == 0:
        ret, frame = cap.read()
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    else:
        rgb = np.array(sct.grab(monitor))
        rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)

    obuff=cv2.resize(rgb, (64, 64))#, interpolation = cv2.INTER_NEAREST)
    ret, obuff = cv2.threshold(obuff, 128, 255, cv2.THRESH_BINARY)
    #print(obuff)
    #break
    
    nobuff = bytearray(int(64*64/8))  # bitmap picture, 1 Byte = 8 pixels
    cbit = 0
    for i in range(64*64):
        bout = obuff[i//64][i%64] > 0
        if bout:
            nobuff[i//8] |= 1 << (i%8)
            
    #print(binascii.hexlify(nobuff))

    sbuff = bytearray(65)
    off = 0
    for r in range(8):
        sbuff[0] = r
        for i in range(64):
            sbuff[1+i] = nobuff[off+i]
        off += 64

        ln = arduino.write(sbuff)
        arduino.flush()
        print('%s, %s' % (ln, binascii.hexlify(sbuff)))
        time.sleep(0.15)
   
    cv2.imshow('frame', obuff)
    cv2.waitKey(1)
    #time.sleep(2)
    #break


65, b'000080ffff1fc0ffff0080ffff1fc0ffff0000fdff1fc0ffff0000fcff0fc0ffff0000f8ff07e0ffff0000f0ff07c0ffff0000e0ff0180ffff000080790000ffff'
65, b'01000400000000ffff000400000000feff000c00000000feff001c00000000feff000c00000000feff000c000000f0fdff0004000000f8f9ff0000000000f0fbff'
65, b'020000000000f0fbff0000000000f0ffff0000000000e0ffff0000000000e0ffff0000000000e0ffff0000000000e0ffff0000000000f0ffff0000000000e0ffff'
65, b'03000000000080ffff000000000080ffff000000000000feff000000000000fcff000000000000fcff000000000010f8ff000000000010f8ff000000000010f8ff'
65, b'04000000000010f8ff000000000010feff000000000030feff000000000030ffff0000000000e0ffff000000000080fcff000000000000fcff000000000000f8ff'
65, b'05000000000000f8ff000000000000fcff000000000000f4ff000000000000fcff000000000000fcff000000000000fc7f000000000000fcff000000000000fc3f'
65, b'06000000000000fc3f000000000000fe2b000000000000fe0f000000000000fe2f000000000000ff15000000000000ff00000000000000ff000000000000007f00'
65, b'070000000000007b000000000000

65, b'04000000e007e0ffff000000e007e0ffff0000004007e0ffff0000004007e0ffff0000004007e0ffff0000004005e0ffff0000004006e0ffff0000000007e0ffff'
65, b'050000000006e0ffff0000000007e0ffff0000000007e0ffff0000000006e0ffff0000000006e0ffff0000000006e0ffff0000000006c0ffff000000000ec0ffff'
65, b'06000000000ec0ffff000000001ec0ff7f000000003ec0ff5f000000003e80ff6f000000003e80ff3f000000003e00ff1f000000003e00fe3f000000003f00fc0f'
65, b'07000000001f00fd05000000001f00f900000000000f00710000000000070071000000000003000200000000000000000000000000000004000000000000000400'
65, b'0000f01fc0fff0ffff00f01f807ff0ffff00f007007ff0ffff00e007007ee0ffff0040071c3ee0ffff0000033e3ce0ffff0000027f1ce0ffff0000007f08e0ffff'
65, b'01000400fe08e0ffff000600fe01e0ffff000c00fe01e0ffff001c00fc01f0ffff001c00fe03f0ffff000c00fe07e0ffff002400fe07f0ffff000000fe07f0ffff'
65, b'02000000fc07f0ffff000000e007f0ffff000000c007f0ffff000000c007f0ffff000000801ff0ffff000080a00ff0ffff000080c007f0ffff000080e11ff0ffff'
65, b'03000080e117f0ffff000080e317

65, b'00d0ffffffffffffffe0fffff7ffffffffe0ffff87ffffffffc0ffff03fefbffff00ffff03fcffffff00faff03f8ffffff00faff03f0ffffff00fdffc3e1ffffff'
65, b'0100fcfff3c3efffff80ebfff9c7efffff00bffff887efffff00fefff88ffeffff00eefff01ffcffff009efff03ff7ffff003e7de03ff6ffff00b062f07ff4ffff'
65, b'02000136f87ff4ffff000000f87ff4ffff000000f0fff4ffff00000080fffcffff00000000fffcffff00000000fefdffff00000000feffffff00000080feffffff'
65, b'0300003f04fefbffff00c01f04fffbffff0000000c7ffbffff0000000effffffff0000009efffdffff0000008efffdffff0000000efffdffff00000008fef4ffff'
65, b'0400000000fcf8ffff000000007cf9ffff0000000078f9ffff0000000078f1ffff0000000078f1ffff0000000078f1ffff0000000070f1ffff0000000070f1ffff'
65, b'050000000070f1ffff0000000070e1ffff0000000070e1ffff0000000078e1ffff0000000078e1ffff0000000070e0ffff0000000070e0ffff0000000070e0ffff'
65, b'0600000000f0c0ffff00000000f0c0ff7f00000000f0c1ff7f00000000e0e1ff7f00000000e083ff5f00000000e003ff1f00000000e003fc1f00000000e003f813'
65, b'0700000000f003e00100000000f0

65, b'04000000000000feff000000000000ffff000000000080ffff000000000080ffff000000000080ffff000000000000ffff000000000000feff000000000000ffff'
65, b'05000000000000ffff000000000000ffff000000000000ffff000000000000ffff000000000000ffff000000000000ffff000000000000ffff000000000080ffff'
65, b'06000000000080ffff000000000080ff7f000000000000ff5f000000000000fc7f000000000000fc3f000000000040f817000000000040e02e0000000000c04004'
65, b'070000000000c000000000000000c000000000000000c000000000000000c000000000000000c000000000000000c0000000000000004000000000000000000000'
65, b'000000feff0fc03f000000feff0fc01f000000fdff07e01f000000f0ff03c01f000000e0ff03e01f000000c03f00e01f000000801000e01f000000000000e01f00'
65, b'010004000000e01f00000a000000e01f00001a000000e00f000018000000e00f000000000000e00f000010000000e00f000000000000e003000000000000e00300'
65, b'020000000000e001000000000000e001000000000000e001000000000000e001000000000000e001000000000000e001000000000000e001000000000000e00100'
65, b'030000000000e001000000000000

KeyboardInterrupt: 

In [ ]:
cv2.destroyAllWindows()

In [ ]:
#w, h = 200, 200
#monitor = {'top': 0, 'left': 0, 'width': w, 'height': h}
#img = Image.frombytes('RGB', (w,h), sct.grab(monitor).rgb)
#cv2.imshow('frame', np.array(img))
cv2.imshow('frame', np.array(sct.grab(monitor)))
if cv2.waitKey(0) & 0xFF == ord('q'):
    cv2.destroyAllWindows()
    

In [ ]:
ret, frame = cap.read()
rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)
print(rgb[0][0])

In [ ]:
cap.release()

In [ ]:
while(True):
#for i in range(2):
    ret, frame = cap.read()
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)
    W = 200
    roff = random.randrange((720-W)*(720-W))
    for r in range(W):
        for q in range(W):
            rgb[(roff//720)+r][(roff%720)+q] = [0x0, 0xff, 0x00, 0xff]
    
    ob = cv2.resize(rgb, (64,64))
    cv2.imshow('image', random.randrange(3,10)*rgb)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        out = cv2.imwrite('capture.jpg', frame)
        break

cap.release()
cv2.destroyAllWindows()
   


In [ ]:
import serial, time
arduino = serial.Serial('/dev/cu.SLAB_USBtoUART', 115200, timeout=.1)

In [ ]:
buff=bytearray(65)

In [ ]:
arduino.write(b'\x04'+buff)

In [ ]:
for i in range(65):
    buff[i] = i

In [ ]:
for j in range(10):
    for i in range(10):
        buff[0] = i
        ln = arduino.write(buff)
        print('%s' % ln)
        time.sleep(0.1)


In [13]:
while True:
    print(arduino.readline())

b'\x80\x80\x00\x80\x00\x80\x80\x80\x00\x80\x00\x00\x00\x80\x00\x80\x80\x80\x80\x80\x00\x80\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x80\x80\x80\x00\x80\x80\x00\x80\x80\x00\x00\x80\x00\x80\x80\x00\x00\x80\x00\x00\x00\x80\x80\x00\x80\x00\x80\x80\x00\x80\x00\x00\x00\x00\x80\x80\x00\x00\x00\x00\x00\x80\x00\x00\x80\x80\x80\x00\x80\x80\x80\x80\x80\x80\x80\x00\x80\x00\x00\x80\x80\x80\x00\x80\x00\x00\x80\x80\x80\x80\x00\x00\x80\x80\x80\x80\x00\x80\x80\x00\x80\x80\x80\x80\x00\x80\x00\x80\x80\x80\x80\x00\x80\x00\x00\x80\x00\x00\x00\x80\x80\x80\x80\x80\x80\x80\x00\x80\x80\x00\x80\x00\x00\x00\x00\x80\x80\x00\x00\x80\x00\x00\x00\x00\x00\x80\x00\x80\x00\x80\x80\x00\x80\x80\x80\x00\x00\x80\x80\x80\x80\x00\x00\x80\x80\x00\x80\x80\x00\x80\x80\x00\x80\x80\x80\x80\x80\x80\x00\x00\x80\x00\x00\x80\x80\x00\x80\x80\x00\x80\x80\x00\x00\x80\x80\x80\x80\x00\x00\x80\x80\x00\x80\x80\x00\x80\x00\x80\x80\x

KeyboardInterrupt: 